In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#読み込み
train_df = pd.read_csv("../input/titanic/train.csv")
test_df = pd.read_csv("../input/titanic/test.csv")

In [ ]:
#テストにサバイブ列（NaN）追加
test_df['Survived'] = np.nan

In [ ]:
#二つ合体
df = pd.concat([train_df, test_df], ignore_index=True, sort=False)

In [ ]:
#ランダムフォレストをインポート
from sklearn.ensemble import RandomForestRegressor

In [ ]:
#Ageとか抜き出し
age_df = df[['Age','Pclass','Sex','Parch','SibSp']]

In [ ]:
#Pclassとかワンホット
age_df=pd.get_dummies(age_df)

In [ ]:
#ヌルとちゃうやつ分ける
known_age = age_df[age_df.Age.notnull()].values  
unknown_age = age_df[age_df.Age.isnull()].values

In [ ]:
#yが'Age'、Xが'Pclass','Sex','Parch','SibSp'
X = known_age[:, 1:]  
y = known_age[:, 0]

In [ ]:
#ランダムフォレスト
rfr = RandomForestRegressor(random_state=0, n_estimators=100, n_jobs=-1)

In [ ]:
#学習
rfr.fit(X, y)

In [ ]:
#予測
predictedAges = rfr.predict(unknown_age[:, 1::])

In [ ]:
#予測Ageはめ込み
df.loc[(df.Age.isnull()), 'Age'] = predictedAges 

In [ ]:
#マップで見てみる
import seaborn as sns
import matplotlib.pyplot as plt
facet = sns.FacetGrid(df[0:890], hue="Survived",aspect=2)
facet.map(sns.kdeplot,'Age',shade= True)
facet.set(xlim=(0, df.loc[0:890,'Age'].max()))
facet.add_legend()
plt.show()

In [ ]:
#Fareのヌルの行
df[df['Fare'].isnull()]

In [ ]:
#EmbarkedがSでPclassが3の行のfareの中央値
fare=df.loc[(df['Embarked'] == 'S') & (df['Pclass'] == 3), 'Fare'].median()

In [ ]:
#ヌルにはめ込む
df['Fare']=df['Fare'].fillna(fare)

In [ ]:
#使う列の指定
df = df[['Survived','Pclass','Sex','Age','Fare','Embarked','SibSp','Parch']]

In [ ]:
#ワンホット
df = pd.get_dummies(df)

In [ ]:
#dfのをtrainとtestに分ける
#合体の時に追加してたテストのSurvived列を消す
train = df[df['Survived'].notnull()]
test = df[df['Survived'].isnull()].drop('Survived',axis=1)

In [ ]:
#Values関数でNumpy配列に
#Numpy配列のほうが計算早い
X = train.values[:,1:]  
y = train.values[:,0] 
test_x = test.values

In [ ]:
#ランダムフォレストとかインポート
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate

In [ ]:
#使う列（特徴）の数
select = SelectKBest(k = 8)

In [ ]:
#ランダムフォレスト
clf = RandomForestClassifier(random_state = 10, 
                             warm_start = True,  
                             n_estimators = 26,
                             max_depth = 6, 
                             max_features = 'sqrt')

In [ ]:
#パイプラインでランダムフォレストまで一気に
pipeline = make_pipeline(select, clf)

In [ ]:
#パイプライン学習
pipeline.fit(X, y)

In [ ]:
#結果見てみる
#Cross Validationはモデルがどれだけ正確か測る
cv_result = cross_validate(pipeline, X, y, cv= 10)
print('mean_score = ', np.mean(cv_result['test_score']))
print('mean_std = ', np.std(cv_result['test_score']))

In [ ]:
#使った行（特徴）どこか
mask= select.get_support()

In [ ]:
#行（特徴）のリスト
list_col = list(df.columns[1:])

In [ ]:
#合わせて使った行一覧表
for i, j in enumerate(list_col):
    print('No'+str(i+1), j,'=',  mask[i])

In [ ]:
#使う行で抜き出し
X_selected = select.transform(X)
print('X.shape={}, X_selected.shape={}'.format(X.shape, X_selected.shape))

In [ ]:
#PassengerId取り出し
PassengerId=test_df['PassengerId']

In [ ]:
#test_xを予測
predictions = pipeline.predict(test_x)

In [ ]:
#提出物データフレーム作成
submission = pd.DataFrame({"PassengerId": PassengerId, "Survived": predictions.astype(np.int32)})

In [ ]:
#csv作成
submission.to_csv("my_submission.csv", index=False)